# Example Firefly usage

This builds off of the [minimal example](https://firefly.rcs.northwestern.edu/docs/data_reader/minimal_example.html) from our docs and the new [data selection example](https://github.com/ageller/Firefly/blob/main/src/firefly/ntbks/selecting_data.ipynb) from our GitHub repo.


As an example, we will generate two random particle sets (a sphere and a cube) both with a randomized attribute field.  We will load these data into Firefly.

This notebook also demonstrates how to retrieve data that you selected by hand in the Firefly app and then use those data with your notebook. 

In [ ]:
%pip install firefly

In [1]:
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Load your data. Here we're creating random data.

# one particle set (sphere)
coords_sphere = np.random.randn( 20000, 3 )
fields_sphere = np.random.random(size=coords_sphere[:,0].size)

# a second particle set (cube)
coords_cube = np.random.rand( 20000, 3 ) - 0.5
fields_cube = np.random.random(size=coords_cube[:,0].size)

## Format Data for Firefly

Here are the docs for [Reader](https://firefly.rcs.northwestern.edu/docs/reference/api/classes/firefly.data_reader.Reader.html#firefly.data_reader.Reader) and [ParticleGroup](https://firefly.rcs.northwestern.edu/docs/reference/api/classes/firefly.data_reader.ParticleGroup.html#firefly.data_reader.ParticleGroup) and also [a tutorial for using these](https://firefly.rcs.northwestern.edu/docs/data_reader/reader_tutorial.html).  Also note that for simple data (e.g., one particle group), you can use the [ArrayReader](https://firefly.rcs.northwestern.edu/docs/reference/api/readers/firefly.data_reader.ArrayReader.html) class.

In [3]:
from firefly.data_reader import ParticleGroup, Reader

In [4]:
# create two Firefly particle groups
sphere = ParticleGroup(
    'sphere',
    coords_sphere,
    field_arrays=[fields_sphere], 
    field_names=['rands']
) 

cube = ParticleGroup(
    'cube',
    coords_cube,
    field_arrays=[fields_cube], 
    field_names=['randc']
) 

# create the Reader
my_reader = Reader()

# add the particle groups to the reader
my_reader.addParticleGroup(sphere)
my_reader.addParticleGroup(cube)


Make sure each field_array (1) has a field_filter_flag (0), assuming True.
Make sure each field_array (1) has a field_colormap_flag (0), assuming True.
Make sure each field_array (1) has a field_radius_flag (0), assuming False.
Make sure each field_array (1) has a field_filter_flag (0), assuming True.
Make sure each field_array (1) has a field_colormap_flag (0), assuming True.
Make sure each field_array (1) has a field_radius_flag (0), assuming False.
datadir is None, defaulting to /opt/anaconda3/lib/python3.9/site-packages/firefly/static/data/Data


## Display Inline

In [5]:
from firefly.server import spawnFireflyServer,quitAllFireflyServers

In [6]:
# define the local port (typically anything in 5000 - 8000 range)
port = 5500

In [7]:
process = spawnFireflyServer(port, method = 'flask')

Waiting up to 10 seconds for background Firefly server to start.....Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
..Launching Firefly at: http://localhost:5500
from directory /opt/anaconda3/lib/python3.9/site-packages/firefly


 * Restarting with watchdog (fsevents)


.Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
.Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
.Launching Firefly at: http://localhost:5500
from directory /opt/anaconda3/lib/python3.9/site-packages/firefly
done! Your server is available at - http://localhost:5500


In [8]:
from IPython.display import IFrame
url = f'http://localhost:{port:d}/combined'
IFrame(url, width=1000, height=500)

In [12]:
# Send data to the server.
# Wait until it loads to run this command
my_reader.sendDataViaFlask()

sphere - 20000/20000 particles - 1 tracked fields
cube - 20000/20000 particles - 1 tracked fields
Posting data on port 5500...======= receiving data from server ...
======= size of data 2123934
======= showing loader
======= sending data to viewer ...
/opt/anaconda3/lib/python3.9/site-packages/firefly/static/data/Data/Datasphere000.json 1060782
/opt/anaconda3/lib/python3.9/site-packages/firefly/static/data/Data/Datacube000.json 1061220
/opt/anaconda3/lib/python3.9/site-packages/firefly/static/data/Data/DataSettings.json 1019
/opt/anaconda3/lib/python3.9/site-packages/firefly/static/data/Data/filenames.json 131
/opt/anaconda3/lib/python3.9/site-packages/firefly/static/data/startup.json 45
======= done
data posted!


## Get the selected data in Python

(after using the selection tool)

In [13]:
import requests

import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
# send a get request to receive the current settings from Firefly
# for larger amounts of data, you will need to increase the waitTime (in seconds) via params (see below; the default is 10s)
r = requests.get(url = f'http://localhost:{port:d}/get_selected_data', params = {'waitTime':60})
if r.status_code == 200:
    # success
    selection = r.json()
    # as a check
    partsKeys = list(selection.keys())
    print(partsKeys)
    print(selection[partsKeys[0]]['Coordinates_flat'][:100]) 
else:
    print('Error: {}'.format(r.status_code), r.content)


======= received request for selected data from user
======= gettings selected data, waiting 60s
!!!!!!!!!!!!!!! TIMEOUT
Error: 504 b'Timeout.  Please increase the waitTime using the params keyword'


In [16]:
# plot x, y for the selected points
partsKeys = list(selection.keys())
part0 = selection[partsKeys[0]]

# UPDATE THIS
x = part0['Coordinates_flat'][0::3]
y = part0['Coordinates_flat'][1::3]

f, ax = plt.subplots()
ax.scatter(x[:1000],y[:1000])

NameError: name 'selection' is not defined

 * Detected change in '/opt/anaconda3/lib/python3.9/site-packages/hdbscan/__init__.py', reloading
 * Detected change in '/opt/anaconda3/lib/python3.9/site-packages/hdbscan/__init__.py', reloading
 * Detected change in '/opt/anaconda3/lib/python3.9/site-packages/hdbscan/branches.py', reloading
 * Detected change in '/opt/anaconda3/lib/python3.9/site-packages/hdbscan/branches.py', reloading
 * Detected change in '/opt/anaconda3/lib/python3.9/site-packages/hdbscan/flat.py', reloading
 * Detected change in '/opt/anaconda3/lib/python3.9/site-packages/hdbscan/flat.py', reloading
 * Detected change in '/opt/anaconda3/lib/python3.9/site-packages/hdbscan/hdbscan_.py', reloading
 * Detected change in '/opt/anaconda3/lib/python3.9/site-packages/hdbscan/hdbscan_.py', reloading
 * Detected change in '/opt/anaconda3/lib/python3.9/site-packages/hdbscan/plots.py', reloading
 * Detected change in '/opt/anaconda3/lib/python3.9/site-packages/hdbscan/plots.py', reloading
 * Detected change in '/opt/anacon

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Launching Firefly at: http://localhost:5500
from directory /opt/anaconda3/lib/python3.9/site-packages/firefly
======= socket connected
======= connected {'data': 'GUI connected!'}
======= connected {'data': 'Viewer connected!'}
======= in room default_Firefly_AMG_ABG
======= in room default_Firefly_AMG_ABG
======= socket connected
======= connected {'data': 'GUI connected!'}
======= connected {'data': 'Viewer connected!'}
======= in room default_Firefly_AMG_ABG
======= in room default_Firefly_AM

### Quit the Firefly server

... this doesn't always work in a notebook ... you can also quit the server by resetting the kernel.

In [ ]:
return_code = quitAllFireflyServers()